CITE Data
CITE gene expresssion (GEX)
CITE antibody derived tag protein (ADT)

# ADT to GEX

In [1]:
import numpy as np
import pandas as pd
import logging
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import os
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt

In [5]:
cite_adt = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_train_mod1.h5ad")
cite_gex = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_train_mod2.h5ad")
cite_sol = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_train_sol.h5ad")

In [6]:
print(f"The training GEX data has {cite_adt.n_obs} observations and {cite_adt.n_vars} features.")
print(f"The training ADT data has {cite_gex.n_obs} observations and {cite_gex.n_vars} features.")
print(f"The training solution data has shape {cite_sol.shape}.")

The training GEX data has 29077 observations and 134 features.
The training ADT data has 29077 observations and 13953 features.
The training solution data has shape (29077, 29077).


In [9]:
cite_adt_test= ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_test_mod1.h5ad")
cite_gex_test = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_test_mod2.h5ad")
cite_sol_test = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_test_sol.h5ad")

In [11]:
print(f"The testing ADT data has {cite_adt_test.n_obs} observations and {cite_adt_test.n_vars} features.")
print(f"The testing GEX data has {cite_gex_test.n_obs} observations and {cite_gex_test.n_vars} features.")
print(f"The testing solution data has shape {cite_sol_test.shape}.")

The testing ADT data has 14813 observations and 134 features.
The testing GEX data has 14813 observations and 13953 features.
The testing solution data has shape (14813, 14813).


In [15]:
adt_train_array=cite_adt.X.toarray()
print(adt_train_array.shape)
print(adt_train_array[1])
print(adt_train_array[1].shape)

(29077, 134)
[1.2411374  0.27233985 0.9930009  0.83153933 0.7073151  2.604268
 2.9432302  0.33823583 0.9065344  1.0725824  0.27233985 0.
 0.8508219  0.04374797 1.3853375  1.994714   2.60757    1.9056884
 1.1880476  0.04374797 0.83153933 1.5308796  0.08566201 1.1027225
 2.926597   1.0255973  0.08566201 0.08566201 0.6388918  0.66222364
 0.48616377 0.42959055 0.33823583 2.3580074  0.3058305  0.04374797
 0.88830626 0.16456151 0.5397071  1.3964658  0.16456151 0.6150026
 0.16456151 0.3058305  1.5405078  0.20179333 0.5654407  0.48616377
 0.9930009  0.1258897  0.7504607  2.1977472  0.40005666 1.9123169
 0.1258897  1.1319807  1.303766   1.5594906  1.8152623  0.6150026
 0.72912055 0.27233985 0.3696239  0.27233985 1.9575262  1.2281269
 1.471078   0.72912055 0.08566201 3.5211635  0.40005666 1.0255973
 0.23768856 1.3853375  0.1258897  0.         1.0255973  0.33823583
 0.9930009  0.3696239  1.8079556  0.16456151 1.6748033  2.2509353
 0.3058305  0.4582772  0.83153933 0.9065344  0.16456151 0.88830626


In [33]:
cite_gex_test.layers["counts"].toarray().shape

(14813, 13953)

In [32]:
cite_gex_test.obs

""
cell_1
cell_2
cell_3
cell_4
cell_5
...
cell_14809
cell_14810
cell_14811
cell_14812


In [6]:
gex_train_array=cite_gex.X.toarray()
print(gex_train_array.shape)
print(gex_train_array[1].shape)

(29077, 13953)
(13953,)


In [6]:
cite_adt.var

,feature_types
CD86,ADT
CD274,ADT
CD270,ADT
CD155,ADT
CD112,ADT
...,...
HLA-E,ADT
CD82,ADT
CD101,ADT
CD88,ADT


In [20]:
cite_gex.obs

,size_factors
0,0.356535
1,1.292643
2,0.970558
3,1.232604
4,0.044585
...,...
29072,0.647419
29073,1.296868
29074,0.757679
29075,3.309557


In [30]:
cite_gex_test.X

<14813x13953 sparse matrix of type '<class 'numpy.float32'>'
	with 23026336 stored elements in Compressed Sparse Column format>

In [12]:
cite_adt.obs

,batch
GCATTAGCATAAGCGG-1-s1d1,s1d1
TACAGGTGTTAGAGTA-1-s1d1,s1d1
AGGATCTAGGTCTACT-1-s1d1,s1d1
GTAGAAAGTGACACAG-1-s1d1,s1d1
TCCGAAAAGGATCATA-1-s1d1,s1d1
...,...
TGTCCCAGTTAACAGA-1-s3d6,s3d6
AATTCCTTCCTAGCCT-1-s3d6,s3d6
CTCAAGAAGATCGCCC-1-s3d6,s3d6
TTGCCTGTCCCATACC-1-s3d6,s3d6


In [21]:
cite_adt_test.obs

,batch
GTAGCTACATCGCTCT-1-s1d2,s1d2
ATCGTAGCACAGCCAC-1-s1d2,s1d2
TCAGTTTGTTGCTGAT-1-s1d2,s1d2
AACCATGAGCTCGAAG-1-s1d2,s1d2
CATTGAGTCTGAATCG-1-s1d2,s1d2
...,...
GTGTAACCATTAAGCC-1-s3d7,s3d7
CAGCCAGCAACGCATT-1-s3d7,s3d7
TCAGCAAAGGTCGTAG-1-s3d7,s3d7
ATCACAGTCGCCAACG-1-s3d7,s3d7


In [11]:
cite_gex.obs

,size_factors
0,0.356535
1,1.292643
2,0.970558
3,1.232604
4,0.044585
...,...
29072,0.647419
29073,1.296868
29074,0.757679
29075,3.309557


In [123]:
cite_gex.uns['dataset_id'] 
cite_adt.uns['dataset_id'] 

'openproblems_bmmc_cite_phase1_MM_adt2gex'

In [124]:
cite_sol_array=adata_sol.X.toarray()
print(cite_sol_array.shape)
print(cite_sol_array[1,1:20])

(29077, 29077)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [126]:
cite_sol_test_array=adata_sol_test.X.toarray()
print(adata_sol_test_array.shape)
print(adata_sol_test_array[1,1:20])

(14813, 14813)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [1]:
cite_gex.obs

NameError: name 'cite_gex' is not defined

In [128]:
cite_sol_test.obs

,batch
GTAGCTACATCGCTCT-1-s1d2,s1d2
ATCGTAGCACAGCCAC-1-s1d2,s1d2
TCAGTTTGTTGCTGAT-1-s1d2,s1d2
AACCATGAGCTCGAAG-1-s1d2,s1d2
CATTGAGTCTGAATCG-1-s1d2,s1d2
...,...
GTGTAACCATTAAGCC-1-s3d7,s3d7
CAGCCAGCAACGCATT-1-s3d7,s3d7
TCAGCAAAGGTCGTAG-1-s3d7,s3d7
ATCACAGTCGCCAACG-1-s3d7,s3d7


# GEX TO ADT

In [97]:
rna_gex = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_train_mod1.h5ad")
rna_adt = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_train_mod2.h5ad")
rna_sol = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_train_sol.h5ad")

In [96]:
print(f"The training RNA data has {rna_gex.n_obs} observations and {rna_gex.n_vars} features.")
print(f"The training RNA data has {rna_adt.n_obs} observations and {rna_adt.n_vars} features.")
print(f"The training solution data has shape {rna_sol.shape}.")

The training RNA data has 29077 observations and 134 features.
The training RNA data has 29077 observations and 13953 features.
The training solution data has shape (29077, 29077).


In [98]:
rna_adt.var

,feature_types
CD86,ADT
CD274,ADT
CD270,ADT
CD155,ADT
CD112,ADT
...,...
HLA-E,ADT
CD82,ADT
CD101,ADT
CD88,ADT


In [129]:
rna_gex_test = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_test_mod1.h5ad")
rna_adt_test = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_test_mod2.h5ad")
rna_sol_test = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_test_sol.h5ad")

In [130]:
print(f"The testing GEX data has {rna_gex_test.n_obs} observations and {rna_gex_test.n_vars} features.")
print(f"The testing ADT data has {rna_adt_test.n_obs} observations and {rna_adt_test.n_vars} features.")
print(f"The testing solution data has shape {rna_sol_test.shape}.")

The testing GEX data has 14813 observations and 13953 features.
The testing ADT data has 14813 observations and 134 features.
The testing solution data has shape (14813, 14813).


In [131]:
rna_gex_array=rna_gex.X.toarray()
print(rna_gex_array.shape)
print(rna_gex_array[1])

(29077, 13953)
[0. 0. 0. ... 0. 0. 0.]


In [134]:
rna_adt_array=rna_adt.X.toarray()
print(rna_adt_array.shape)
print(rna_adt_array[1])

(29077, 134)
[0.         0.70202017 0.857432   0.         0.99190825 0.99190825
 0.99190825 0.2921284  1.5543532  0.2921284  0.5179303  0.5179303
 1.5543532  0.         0.         0.         1.2163731  2.094918
 0.5179303  2.094918   0.857432   0.857432   0.         0.2921284
 0.857432   0.2921284  0.         0.         1.2163731  1.1104255
 0.99190825 0.70202017 0.2921284  0.         0.70202017 0.
 0.2921284  0.         0.5179303  0.2921284  0.5179303  0.
 0.2921284  0.2921284  0.         0.5179303  0.857432   0.70202017
 0.2921284  0.         0.5179303  1.5543532  0.         0.2921284
 0.2921284  0.         0.         1.9610732  0.70202017 1.9610732
 0.         0.70202017 1.1104255  0.2921284  0.857432   0.
 1.5543532  1.4799554  0.         1.9610732  0.5179303  0.857432
 0.         0.70202017 0.         0.         1.2163731  0.2921284
 0.5179303  1.1104255  0.70202017 0.99190825 0.70202017 0.
 0.99190825 0.857432   1.7491738  0.70202017 0.         1.1104255
 0.5179303  0.5179303  0.

In [132]:
rna_gex.uns['dataset_id']

'openproblems_bmmc_cite_phase1_MM_gex2adt'